In [1]:
import pandas as pd
import numpy as np
from numpy  import array
import librosa
import os
import ntpath
import pathlib
from pathlib import Path
import pickle
from tinytag import TinyTag
from pydub import AudioSegment
from pydub.utils import make_chunks
from itertools import tee, islice, chain
from math import atan2,degrees

In [2]:
def getVocalsAndInstruments(stft,sr):
    S_full, phase = librosa.magphase(stft)
    # We'll compare frames using cosine similarity, and aggregate similar frames
    # by taking their (per-frequency) median value.
    #
    # To avoid being biased by local continuity, we constrain similar frames to be
    # separated by at least 2 seconds.
    #
    # This suppresses sparse/non-repetetitive deviations from the average spectrum,
    # and works well to discard vocal elements.

    S_filter = librosa.decompose.nn_filter(S_full,aggregate=np.median,metric='cosine',width=int(librosa.time_to_frames(2, sr=sr)))

    # The output of the filter shouldn't be greater than the input
    # if we assume signals are additive.  Taking the pointwise minimium
    # with the input spectrum forces this.
    S_filter = np.minimum(S_full, S_filter)
    # We can also use a margin to reduce bleed between the vocals and instrumentation masks.
    # Note: the margins need not be equal for foreground and background separation
    margin_i, margin_v = 2, 10
    power = 2

    mask_i = librosa.util.softmask(S_filter,margin_i * (S_full - S_filter),power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,margin_v * S_filter,power=power)

    # Once we have the masks, simply multiply them with the input spectrum
    # to separate the components
    #S_foreground 
    return (S_full,(mask_i * S_full),(mask_v * S_full))

In [3]:
def generateDataFrameWithHeader(header):
    return pd.DataFrame(columns=header)

def appendMetadata(songname):
    featurelist = []
    tag = TinyTag.get(songname)
    featurelist.append(ntpath.basename((songname)))
    featurelist.append(tag.artist)
    featurelist.append(tag.duration)
    featurelist.append(tag.albumartist)
    featurelist.append(tag.bitrate)
    featurelist.append(tag.comment)
    featurelist.append(tag.filesize)
    featurelist.append(tag.genre)
    featurelist.append(tag.title) 
    featurelist.append(tag.track) 
    featurelist.append(tag.samplerate) 
    featurelist.append(tag.track_total) 
    featurelist.append(tag.year)
    return featurelist

def get_Wav_from_Mp3(songname):
    sound = AudioSegment.from_mp3(songname)
    filename = "final.wav"
    sound.export(filename, format="wav")
    return filename



In [4]:
def getAudioInformation(filename):
    featurelist = []
    y, sr = librosa.load(filename, mono=True)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    full_spectrum,instruments,vocals = getVocalsAndInstruments(librosa.stft(y),sr)
    tempo, beat_times = librosa.beat.beat_track(y)
###-----------------------Audio Metadata-----13-----------------------------
    featurelist = appendMetadata(songname)
###-----------------------Audio Features----------------------28------------
    featurelist.append(pickle.dumps(y,protocol = -1) )
    featurelist.append(sr)
    featurelist.append(np.mean(full_spectrum))
    featurelist.append(np.mean(instruments))
    featurelist.append(np.mean(vocals))
    featurelist.append(pickle.dumps(chroma_stft,protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.spectral_centroid(y=y, sr=sr),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.spectral_bandwidth(y=y, sr=sr),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.spectral_rolloff(y=y, sr=sr),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.zero_crossing_rate(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.mfcc(y=y, sr=sr),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.rmse(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.chroma_cqt(y , sr=sr),protocol = -1)) 
    featurelist.append(pickle.dumps(librosa.feature.chroma_cens(y , sr=sr),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.melspectrogram(y, sr=sr),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.spectral_contrast(y , sr=sr),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.spectral_flatness(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.poly_features(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.tonnetz(y),protocol = -1))
    tempogram1 = librosa.feature.tempogram(y)
    featurelist.append(pickle.dumps(tempogram1,protocol = -1))
    featurelist.append(pickle.dumps(librosa.onset.onset_detect(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.onset.onset_strength(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.onset.onset_strength_multi(y),protocol = -1))
    featurelist.append(pickle.dumps(beat_times,protocol = -1))
    featurelist.append(pickle.dumps(tempo,protocol = -1))
    featurelist.append(pickle.dumps(librosa.effects.hpss(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.effects.percussive(y),protocol = -1))
    featurelist.append(pickle.dumps(librosa.feature.delta(tempogram1),protocol = -1))
    
    return featurelist

In [5]:
header = 'filename artist duration albumartist bitrate comment filesize genre title track samplerate track_total year audio_waveform sampling_rate full_spectrum_mean instruments_mean vocals_mean '
fe_col_header = 'chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate mfcc rmse chroma_cqt chroma_cens melspectrogram spectral_contrast spectral_flatness poly tonal_centroid tempogram onset_detect onset_strength onset_strength_multi beat tempo harmonic_elements percussive_elements delta'
header += fe_col_header
header = header.split()

In [6]:
def extractInfo(directory_path,storage_path):
    pathlist = Path(directory_path).glob('**/*.mp3')
    df = generateDataFrameWithHeader(header)
    sr_no = 0
    for path in pathlist:
        sr_no += 1
        path_in_str = str(path)
        songname = path_in_str.replace(os.sep, '/')

        ###-----List to store all the metadata and other information for an audio file------
        featurelist = []

        ###-----MP3 file conersion to wave format for better analysis-------------
        filename = get_Wav_from_Mp3(songname)

        ###----------------storing  the audio file information --------------------
        featurelist =  getAudioInformation(songname)

        ###-------Storing the information-list  to  Pandas DataFarme--------------------
        df.loc[sr_no] = featurelist
    df.to_pickle(storage_path,protocol = -1)
    print(df.head())

In [7]:
mp3_audio_directory_path = "D:/learning/DataScience/data/musicFiles" 
pickle_file_storage_path = "D:/learning/DataScience/data/prj/featureExtraction/picklle_file/audio_information_extract_april20.pkl"


In [ ]:
extractInfo(mp3_audio_directory_path,pickle_file_storage_path)